In [7]:
import pandas as pd
import numpy as np
import plot_results as pr
import matplotlib.pyplot as plt
from matplotlib import ticker
import h5py
import os
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['axes.facecolor'] = 'w'

### Model parameters

In [8]:
## Define compartments
# Use this dict for new model outputs
secir_dict = {0: 'SusceptibleNaive', 1: 'SusceptiblePartialImmunity', 2: 'ExposedNaive', 3: 'ExposedPartialImmunity', 
                4: 'ExposedImprovedImmunity', 5: 'InfectedNoSymptomsNaive', 6: 'InfectedNoSymptomsPartialImmunity', 
                7: 'InfectedNoSymptomsImprovedImmunity ', 8: 'InfectedNoSymptomsNaiveConfirmed', 9: 'InfectedNoSymptomsPartialImmunityConfirmed', 
                10: 'InfectedNoSymptomsImprovedImmunityConfirmed', 11: 'InfectedSymptomsNaive', 12: 'InfectedSymptomsPartialImmunity', 
                13: 'InfectedSymptomsImprovedImmunity', 14: 'InfectedSymptomsNaiveConfirmed', 15: 'InfectedSymptomsPartialImmunityConfirmed',
                16: 'InfectedSymptomsImprovedImmunityConfirmed', 17: 'InfectedSevereNaive', 18: 'InfectedSeverePartialImmunity',
                19: 'InfectedSevereImprovedImmunity', 20: 'InfectedCriticalNaive', 21: 'InfectedCriticalPartialImmunity', 
                22: 'InfectedCriticalImprovedImmunity', 23: 'SusceptibleImprovedImmunity', 24: 'DeadNaive', 25: 'DeadPartialImmunity',
                26: 'DeadImprovedImmunity', 25: 'TotalInfections'}
# Use this dict for model outputs of working branch before mid of 2022                                
#secir_dict = {0:'Susceptible', 1:'Partially Vaccinated', 2:'Exposed', 3:'ExposedV1', 4:'ExposedV2', 5:'Carrier',
#              6:'CarrierV1',7:'CarrierV2', 8:'CarrierT', 9:'CarrierTV1', 10:'CarrierTV2', 11:'Infected',
#              12:'InfectedV1',13:'InfectedV2', 14:'InfectedT', 15:'InfectedTV1', 16:'InfectedTV2',
#              17:'Hospitalized', 18:'HospitalizedV1', 19:'HospitalizedV2', 20:'ICU',
#              21:'ICUV1', 22:'ICUV2', 23:'Immune', 24:'Dead', 25: 'Infected Total'}

# Define aggregated infection states to plot and create 'concat_comps' dictionary for aggregation
secir_dict_aggregated = {0:'Susceptible', 1:'Partial Immunity', 2:'Exposed', 3:'InfectedNoSymptoms', 4:'InfectedSymptoms',
              5:'InfectedSevere', 6:'InfectedCritical', 7:'Improved Immunity', 8:'Dead', 9: 'Infected Total'}
#concat_comps = {0:[0], 1:[1], 7:[23], 8:[24], 9:[25]} # old, only one dead compartment (need to change secir dict above)
concat_comps = {0:[0], 1:[1], 7:[23], 8:[24, 25, 26], 9:[27]} # new, three dead compartments
for i in range(2,7):
    concat_comps[i] = []
    for k, infstate in secir_dict.items():
        if secir_dict_aggregated[i] in infstate:
            if (infstate != 'TotalInfections') and (infstate != 'Infected Total'):
                concat_comps[i].append(k)


# Define age groups
age_groups = ['0-4 Years', '5-14 Years', '15-34 Years', '35-59 Years', '60-79 Years', '80+ Years']

# Define population data for incidence values and relative plots
base = 100000
age_group_sizes = np.array([3961376,7429883,19117865,28919134,18057318,5681135])

relative_dict = {}
for i in range(len(age_group_sizes)):
    relative_dict['Group' + str(i+1)] = age_group_sizes[i]/base
    
relative_dict['Total'] = np.sum(age_group_sizes)/base   

### Simulation parameters 

In [11]:
# Define start day and simulation period
year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 45

# Define different scenario folders that will be read and plotted
date_str = '_' + str(year) + '_' + str(month) + '_' + str(day) + '_' + str(tmax)
path_sim = 'data/'
path_rki = 'data/'
scenario_list = ['', '_long', '_long_high', '_high', '_late', '_mask_test', 
                 '_high_late', '_high_mask_test', '_late_mask_test','_high_late_mask_test']

# Provide a list of labels for corresponding plots
scenario_label = {
    '': 'No commuter testing, local NPIs decreed until July 1st, no masks after opening' ,
    '_late': 'No commuter testing, local NPIs decreed until August 1st, no masks after opening' ,
    '_mask_test': 'Commuter testing, local NPIs decreed until July 1st, keeping masks and distancing', 
    '_late_mask_test': 'Commuter testing, local NPIs decreed until August 1st, keeping masks and distancing',
    '_long': 'No commuter testing, local NPIs decreed until July 1st, no masks after opening' ,
    '_long_late': 'No commuter testing, local NPIs decreed until August 1st, no masks after opening' ,
    '_long_mask_test': 'Commuter testing, local NPIs decreed until July 1st, keeping masks and distancing', 
    '_long_late_mask_test': 'Commuter testing, local NPIs decreed until August 1st, keeping masks and distancing',
    '_future_long': 'No commuter testing, no local NPIs' ,
    '_future_long_mask_test': 'No commuter testing, no local NPIs except for masks and distancing',  
    '_future': 'No commuter testing, no local NPIs' ,
    '_future_mask_test': 'No commuter testing, no local NPIs except for masks and distancing', 
    '_high': 'No commuter testing, local NPIs decreed until July 1st, no masks after opening' ,
    '_high_late': 'No commuter testing, local NPIs decreed until August 1st, no masks after opening' ,
    '_high_mask_test': 'Commuter testing, local NPIs decreed until July 1st, keeping masks and distancing', 
    '_high_late_mask_test': 'Commuter testing, local NPIs decreed until August 1st, keeping masks and distancing',
    '_long_high': 'No commuter testing, local NPIs decreed until July 1st, no masks after opening' ,
    '_long_high_late': 'No commuter testing, local NPIs decreed until August 1st, no masks after opening' ,
    '_long_high_mask_test': 'Commuter testing, local NPIs decreed until July 1st, keeping masks and distancing', 
    '_long_high_late_mask_test': 'Commuter testing, local NPIs decreed until August 1st, keeping masks and distancing',
    '_future_long_high': 'No commuter testing, no local NPIs' ,
    '_future_long_high_mask_test': 'No commuter testing, no local NPIs except for masks and distancing',  
    '_future_high': 'No commuter testing, no local NPIs' ,
    '_future_high_mask_test': 'No commuter testing, no local NPIs except for masks and distancing',
    's1f_future_high_mask_test': 'S1F: contact reduc. 27 % [21-32 %]',
    's2f_future_high_mask_test': 'S2F: contact reduc. 37% [32-42 %]',
    's3f_future_high_mask_test': 'S3F: contact reduc. 42% [37-46]'
}

### Plot parameters

In [10]:
plotRKI = True           # Plots RKI Data if true
plotRelative = False     # Plots incidence values if true
plotPercentiles = True  # Plots 25 and 75 percentiles if true
plotConfidence = False   # Plots 05 and 95 percentiles if true
savePlot = True          # saves plot file if true

if savePlot:
    try:
        os.mkdir('Plots')
    except:
        print('Directory "Plots" already exists')
        
fontsize = 28
figsize = (13, 10)

#define x-ticks for plots
datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())

plt_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

colors = {}
colors['Total'] = plt_colors[0]
for i in range(len(age_groups)):
    colors['Group' + str(i+1)] = plt_colors[i+1]

Directory "Plots" already exists


# Plots For Paper

In [ ]:
# This Cell plots all Scenarios with combined compartments

year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 45

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
plotRKI=True
plotLegend=True
ylim = {}

secir_dict_inv = dict([(val, key) for key, val in secir_dict.items()])
# List of integers corresponding to the compartments to plot
# e.g. comparts = [4, 6, 8] would only plot Infected, ICU and Dead 
for compart, compart_label in secir_dict_aggregated.items():
    #if compart_label == 'Infected':
    for high in ['_high', '']:
        scenario_list = [high, high + '_late', high + '_mask_test', high + '_late_mask_test','_long' +  high + '_mask_test'] 

        files = pr.open_files(spec_str_sim='_rev2', spec_str_rki1 = '', scenario_list = scenario_list, read_casereports_extrapolation = True)
        new_files = pr.concat_comparts(files, concat_comps, scenario_list)
        for scenario in scenario_list:
            if True: # compart_label in ['Infected','ICU','Dead']:
                if compart_label=='Infected':
                    plotLegend=True
                else:
                    plotLegend=False
                if compart_label=='Dead' and plotRKI and 'RKI' in files.keys():
                    addBase = files[scenario]['RKI'][regionid]['Total'][:,secir_dict_inv['Dead']][0]
                else:
                    addBase = 0
                ylim = pr.plot_results(new_files[scenario], compart, secir_dict_aggregated[compart], ylim, filename=scenario, key='Total', plotLegend=plotLegend, addVal=addBase)

        pr.close_files(files)
            


In [ ]:
# Plots for Future 
year, month, day = '2021', '10', '15'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 90

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1
plotRKI=False
plotPercentiles = True
ylim = {}
comparts = [4]
scenario_list = ['s1f_future_high_mask_test', 's2f_future_high_mask_test', 's3f_future_high_mask_test']
for compart in comparts:

    files = pr.open_files(spec_str_sim='_rev2_', spec_str_rki1 = '', spec_str_rki2 = '_future', scenario_list = scenario_list, read_casereports_extrapolation = True)
    new_files = pr.concat_comparts(files, concat_comps, scenario_list)
    pr.plot_all_results(new_files, compart, secir_dict_aggregated[compart], filename=high + '_all', key='Total', show_perc=True)
    for scenario in scenario_list:
        ylim = pr.plot_results(new_files[scenario], compart, secir_dict_aggregated[compart], ylim, filename=scenario, key='Total')

    pr.close_files(files)
        

In [ ]:
# Barplot for all Scenarios except future
scenario_list = ['',  '_late', '_mask_test', '_late_mask_test', '_high',  '_high_late', '_high_mask_test', '_high_late_mask_test']
plotRKI=False
files = pr.open_files(spec_str_sim='_rev2', spec_str_rki1 = '', scenario_list = scenario_list, read_casereports_extrapolation = True)
new_files = pr.concat_comparts(files, concat_comps, scenario_list)


columns = ('S1 - 40%', 'S2 - 40%', 'S3 - 40%', 'S4 - 40%', 'S1 - 60%', 'S2 - 60%', 'S3 - 60%', 'S4 - 60%')
rows =  ['0-4 Years', '5-14 Years', '15-34 Years', '35-59 Years', '60-79 Years', '80+ Years']

pr.plot_bars(True, 'age_incidence', new_files, columns, rows, 200)
pr.close_files(files)

In [ ]:
# Barplot for Future Scenarios
scenario_list = ['s1f_future_high_mask_test', 's2f_future_high_mask_test', 's3f_future_high_mask_test']
files = pr.open_files(spec_str_sim='_rev2_', spec_str_rki1 = '', spec_str_rki2 = '_future', scenario_list = scenario_list, read_casereports_extrapolation = True)
new_files = pr.concat_comparts(files, concat_comps, scenario_list)

fs = 24

columns = ('S1F: contact reduc. 27 % [21-32 %]', 'S2F: contact reduc. 37% [32-42 %]', 'S3F: contact reduc. 42% [37-46]')
rows =  ['0-4 Years', '5-14 Years', '15-34 Years', '35-59 Years', '60-79 Years', '80+ Years']

pr.plot_bars(False, 'age_incidence', new_files, columns, rows, 200)
pr.close_files(files)

In [ ]:
# Plot for Partially Vaccinated, Vaccinated and Cumulative number of Infections
year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 91
regionid = '0'

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1
plotRKI=False
plotPercentiles = False
savePlot = True
ylim = {}
comparts = [1,7, 9]
scenario_list = ['_high_mask_test']

files = files = pr.open_files(spec_str_sim='_rev2', spec_str_rki1 = '', scenario_list = scenario_list)
new_files = pr.concat_comparts(files, concat_comps, scenario_list)
new_files[scenario_list[0]]['p50'][regionid]['Total'][:,7] -= new_files[scenario_list[0]]['p50'][regionid]['Total'][:,9]
temp_secir_dict = {
    1: 'Partially Vaccinated',
    7: 'Fully Vaccinated',
    9: 'Cumulative number of Infections'
}
for compart in comparts:
    
    for scenario in scenario_list:
        pr.plot_results_secirvvs(new_files[scenario], compart, temp_secir_dict[compart], None, filename='vacc' + scenario, key='Total', plotLegend=False)
        
    pr.close_files(files)

# Other Plots (not for paper)

In [ ]:
# This Cell Plots all Scenarios, where the Y limits for "high" and 
# not "high" Scenarios are equal (Without "concat_comparts")
year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 91

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1
plotRKI=False
ylim = {}
comparts = secir_dict
for compart in comparts:
    for high in ['_high', '']:
        scenario_list = [high, high + '_late', high + '_mask_test', high + '_late_mask_test','_long' +  high + '_mask_test'] 
        
        files = pr.open_files('_init_test')
        new_files = pr.concat_comparts(files, concat_comps)
        #plot_all_results(new_files, compart, new_secir_dict[compart], filename=high + '_all', key='Total')
        for scenario in scenario_list:
            ylim = pr.plot_results(files[scenario], compart, secir_dict[compart], ylim, filename=scenario, key='Total')
            
        pr.close_files(files)
            


In [ ]:
# This Cell only plots one Scenario (Without "concat_comparts")
year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 91

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1

comparts = secir_dict  
for compart in comparts:
    # List of strings corresponding to Scenarios
    scenario_list = ['_high_mask_test']

    files = pr.open_files()
    new_files = pr.concat_comparts(files, concat_comps)
    #plot_all_results(new_files, compart, new_secir_dict[compart], filename=high + '_all', key='Total')
    ylim = None
    for scenario in scenario_list:
        ylim = pr.plot_results_secirvvs(files[scenario], compart, secir_dict[compart], ylim, filename=scenario, key='Total')

    pr.close_files(files)
            

In [ ]:
# This Cell plots all Scenarios with combined compartments

year, month, day = '2021', '6', '6'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '90'
daysPlot = 91

datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1
plotRKI=False
ylim = {}
# List of integers corresponding to the compartments to plot
# e.g. comparts = [4, 6, 8] would only plot Infected, ICU and Dead 
comparts = new_secir_dict
for compart in comparts:
    #for high in ['_high', '']:
    #scenario_list = [high, high + '_late', high + '_mask_test', high + '_late_mask_test','_long' +  high + '_mask_test'] 
    scenario_list = ['_high_mask_test']
    files = pr.open_files()
    new_files = pr.concat_comparts(files, concat_comps)
    #plot_all_results(new_files, compart, new_secir_dict[compart], filename=high + '_all', key='Total')
    for scenario in scenario_list:
        ylim = pr.plot_results_secirvvs(new_files[scenario], compart, new_secir_dict[compart], ylim, filename=scenario, key='Total')

    pr.close_files(files)